In [ ]:
%env AIBS_RIG_ID=NP1

In [ ]:
import contextlib
import time
import requests
import yaml
import os
import pathlib
from copy import deepcopy
from uuid import uuid4

import np_config
import np_jobs
import np_logging
import np_services
import np_session
import typing
import np_workflows
from np_workflows import npxc
import np_workflows.experiments.openscope_barcode as Barcode

from np_services.resources.zro import ZroError 
import contextlib

logger = np_logging.getLogger()

np_workflows.elapsed_time_widget()

In [ ]:
user, mouse = np_workflows.user_and_mouse_widget()

In [ ]:
np_workflows.mtrain_widget(mouse)

In [ ]:
# fetch required resources for running these things
def wrapped_get(url: str, **kwargs) -> requests.Response:
    """Wrap requests.get to raise for non-200 status codes."""
    response = requests.get(url, **kwargs)
    if response.status_code != 200:
        response.raise_for_status()
    return response


def find_task_output(session: np_session.Session) -> pathlib.Path:
    pass


mtrain_regimen_response = wrapped_get(
    "https://raw.githubusercontent.com/AllenInstitute/mtrain_regimens/VisualBehaviorEPHYS_Task1G_v0.1.2/regimen.yml",
)
mtrain_regimen = yaml.safe_load(mtrain_regimen_response.text)
task_params = mtrain_regimen["stages"]["EPHYS_1_images_G_3uL_reward"]["parameters"]

commit_hash = '5adfa6e285774719135d0ebcba421f15f6f56168'
# script content to be write to files
behavior_script_content = requests.get(
    f'http://stash.corp.alleninstitute.org/projects/VB/repos/visual_behavior_scripts/raw/replay_session/behavior_script.py?at={commit_hash}',
).text

mapping_script_content = requests.get(
    f'http://stash.corp.alleninstitute.org/projects/VB/repos/visual_behavior_scripts/raw/replay_session/mapping_script.py?at={commit_hash}',
).text

replay_script_content = requests.get(
    f'http://stash.corp.alleninstitute.org/projects/VB/repos/visual_behavior_scripts/raw/replay_session/replay_script.py?at={commit_hash}',
).text

optotagging_script_content = requests.get(
    f'http://stash.corp.alleninstitute.org/projects/VB/repos/visual_behavior_scripts/raw/replay_session/optotagging_script.py?at={commit_hash}',
).text

# use braintv directory for testing
local_behavior_script = pathlib.Path("//allen/programs/braintv/workgroups/nc-ophys/1022/behavior_script.py")
local_behavior_script.write_text(behavior_script_content)
local_mapping_script = pathlib.Path("//allen/programs/braintv/workgroups/nc-ophys/1022/mapping_script.py")
local_mapping_script.write_text(mapping_script_content)
local_replay_script = pathlib.Path("//allen/programs/braintv/workgroups/nc-ophys/1022/replay_script.py")
local_replay_script.write_text(replay_script_content)
local_optotagging_script = pathlib.Path("//allen/programs/braintv/workgroups/nc-ophys/1022/optotagging_script.py")
local_optotagging_script.write_text(optotagging_script_content)

gabor_path = task_params['mapping']['gabor_path']
flash_path = task_params['mapping']['flash_path']

assert os.path.exists(gabor_path) and os.path.exists(flash_path), \
    "mapping stim should exist"

foraging_id = {
    'value': uuid4().hex,
    'inferrred': False,
}
TASK_ID = "replay"
# add additional parameters that help us log better, link these scripts together
task_params['replay_id'] = foraging_id['value']
behavior_params = deepcopy(task_params)
behavior_params['foraging_id'] = foraging_id
behavior_params['task'] = {
    "id": TASK_ID,
    "sub_id": "behavior",
    "scripts_hash": commit_hash,
}

default_levels  = [1.0, 1.2, 1.3] #TODO what should these be?
config_path = r'C:/ProgramData/camstim/config/stim.cfg'  # TODO: make this work, this has probably moved

opto_params = deepcopy(task_params.get("opto_params"))
# opto_params["mouseID"] = task_params["mouse_id"]
# try:
#     stim_cfg_opto_params = get_config(
#         'Optogenetics',
#         path=config_path,
#     )
#     opto_params["level_list"] = stim_cfg_opto_params["level_list"]
#     logger.info(f'Reading opto level list from stim cfg ({config_path}): {stim_cfg_opto_params["level_list"]}')
# except Exception:
#     logger.error(
#         f'Could not get opto level list from stim cfg ({config_path}) using default levels: {default_levels}',
#         exec_info=True,
#     )
opto_params["level_list"] = default_levels


class SevMixin:

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # self.imager = np_services.Imager()
        self._camstim_script_path = local_behavior_script.as_posix()
        self._logging_postfix = "behavior"
        self._camstim_params = behavior_params
        self._camstim_params["output_path"] = self.session.folder

    @property
    def camstim_script_path(self) -> str:
        return self._camstim_script_path
    @camstim_script_path.setter
    def camstim_script_path(self, path: str) -> None:
        self.camstim_script_path = path

    @property
    def logging_postfix(self) -> str:
        return self._logging_postfix
    @logging_postfix.setter
    def logging_postfix(self, value: str) -> None:
        self._logging_postfix = value

    @property
    def camstim_params(self) -> dict[str, typing.Any]:
        return self._camstim_params
    @camstim_params.setter
    def camstim_params(self, value: dict[str, typing.Any]) -> None:
        self._camstim_params = value

    @property
    def recorders(self) -> tuple[np_services.Service, ...]:
        """Services to be started before stimuli run, and stopped after.
         Session-dependent.
        """
        return (np_services.Sync, np_services.VideoMVR, np_services.OpenEphys)

    @property
    def stims(self) -> tuple[np_services.Service, ...]:
        return (np_services.ScriptCamstim, )

    def initialize_and_test_services(self) -> None:
        """Configure, initialize (ie. reset), then test all services."""
        
        np_services.MouseDirector.user = self.user.id
        np_services.MouseDirector.mouse = self.mouse.id

        np_services.OpenEphys.folder = self.session.folder

        np_services.NewScaleCoordinateRecorder.log_root = self.session.npexp_path
        np_services.NewScaleCoordinateRecorder.log_name = self.platform_json.path.name

        assert pathlib.Path(self.camstim_script_path).exists(), f"Filepath does not exist or is not accessible: {self.camstim_script_path=}"
        assert pathlib.Path(self.camstim_params_path).exists(), f"Filepath does not exist or is not accessible: {self.camstim_params_path=}"
        
        np_services.ScriptCamstim.script = self.camstim_script_path
        np_services.ScriptCamstim.params = self.camstim_params
        
        self.configure_services()

        super().initialize_and_test_services()

    def update_state(self) -> None:
        "Store useful but non-essential info."
        self.mouse.state['last_session'] = self.session.id
        # self.mouse.state['last_vbn_session'] = str(self.workflow)
        # self.mouse.state['last_ephys_day'] = self.ephys_day
        today = self.session.date.strftime('%y%m%d')
        # previous_run_date = self.mouse.state.setdefault(f'ephys_day_{self.ephys_day}', today)
        # assert previous_run_date == today, f"{self.ephys_day = } already ran on {previous_run_date}"
            
        if self.mouse == 366122:
            return
        
        self.session.project.state['latest_ephys'] = self.session.id
        self.session.project.state['sessions'] = self.session.project.state.get('sessions', []) + [self.session.id]

    def run_stim(self) -> None:
        self.update_state()
        
        if not np_services.ScriptCamstim.is_ready_to_start():
            raise RuntimeError("ScriptCamstim is not ready to start.")
        
        np_services.ScriptCamstim.script = self.camstim_script_path
        np_services.ScriptCamstim.params = self.camstim_params
        np_logging.web(f'sev_dev_{self.logging_postfix}').info(f"Started camstim script")
        np_services.ScriptCamstim.start()
        
        with contextlib.suppress(Exception):
            while not np_services.ScriptCamstim.is_ready_to_start():
                time.sleep(2.5)
            
        if isinstance(np_services.ScriptCamstim, np_services.Finalizable):
            np_services.ScriptCamstim.finalize()

        with contextlib.suppress(Exception):
            np_logging.web(f'barcode_{self.logging_postfix}').info(f"Finished session {self.mouse.mtrain.stage['name']}")

    def find_task_outputs(self) -> str:
        if None == (stim_pkl := next(self.session.npexp_path.glob(f'{self.session.date:%y%m%d}*_{self.session.mouse}_*.pkl'), None)):
            logger.warning('Did not find stim file on npexp matching the format `YYYYMMDDSSSS_mouseID_foragingID.pkl`')
            return

    def copy_data_files(self) -> None: 
        super().copy_data_files()
        # When all processing completes, camstim Agent class passes data and uuid to
        # /camstim/lims BehaviorSession class, and write_behavior_data() writes a
        # final .pkl with default name YYYYMMDDSSSS_mouseID_foragingID.pkl
        # - if we have a foraging ID, we can search for that
        if None == (stim_pkl := next(self.session.npexp_path.glob(f'{self.session.date:%y%m%d}*_{self.session.mouse}_*.pkl'), None)):
            logger.warning('Did not find stim file on npexp matching the format `YYYYMMDDSSSS_mouseID_foragingID.pkl`')
            return
        assert stim_pkl
        if not self.session.platform_json.foraging_id:
            self.session.platform_json.foraging_id = foraging_id["value"]
        new_stem = f'{self.session.folder}.stim'
        logger.debug(f'Renaming stim file copied to npexp: {stim_pkl} -> {new_stem}')
        stim_pkl = stim_pkl.rename(stim_pkl.with_stem(new_stem))
        
        # remove other stim pkl, which is nearly identical, if it was also copied
        for pkl in self.session.npexp_path.glob('*.pkl'):
            if (
                self.session.folder not in pkl.stem
                and 
                abs(pkl.stat().st_size - stim_pkl.stat().st_size) < 1e6
            ):
                logger.debug(f'Deleting extra stim pkl copied to npexp: {pkl.stem}')
                pkl.unlink()


class Sev(SevMixin, np_workflows.PipelineEphys):
    def __init__(self, *args, **kwargs):
        self.services = (
            np_services.MouseDirector,
            np_services.Sync,
            np_services.VideoMVR,
            self.imager,
            np_services.NewScaleCoordinateRecorder,
            np_services.SessionCamstim,
            np_services.OpenEphys,
        )
        super().__init__(*args, **kwargs)

In [ ]:
with contextlib.suppress(Exception):
    np_services.start_rsc_apps()

## Generate new session

In [ ]:
experiment: np_workflows.PipelineExperiment = Sev(mouse, user)

with contextlib.suppress(Exception):
    np_logging.web(f'barcode_{experiment.workflow.name.lower()}').info(f"{experiment} created")

session: np_session.PipelineSession = experiment.session
platform_json: np_session.PlatformJson = experiment.session.platform_json

platform_json.workflow_start_time = npxc.now()

## Run behavior script

### Extend lick spout before behavior

In [ ]:
np_services.MouseDirector.get_proxy().extend_lick_spout()

In [ ]:
# Params, script, logging for behavior set on init
experiment.camstim_params["mouseID"] = mouse.id
experiment.camstim_params['max_task_duration_min'] = 1  # override max task duration for testing
experiment.run_stim()

## Run mapping script

### Retract lick spout before mapping

In [ ]:
np_services.MouseDirector.get_proxy().retract_lick_spout()

Use task duration for testing

In [ ]:
# max_mapping_duration = task_params.get("max_mapping_duration_min")
max_mapping_duration = task_params['max_task_duration_min']

In [ ]:
experiment.camstim_script_path = local_mapping_script.as_posix()
experiment.camstim_params = {
    'foraging_id': foraging_id,
    'gabor_path': gabor_path,
    'flash_path': flash_path,
    # "output_path": mapping_output_path,
    "mouseID": mouse.id,
    "task": {
        "id": TASK_ID,
        "sub_id": "mapping",
        "scripts_hash": commit_hash,
    },
    "regimen": task_params.get("regimen"),
    "stage": task_params.get("stage"),
    "mouse_id": task_params.get("mouse_id"),
    "max_mapping_duration_min": max_mapping_duration,
}
experiment.logging_prefix = "mapping"
experiment.run_stim()

## Run replay script

In [ ]:
# replace behavior script with replay script
behavior_script_output_path = find_task_output()
replay_params = deepcopy(task_params)
replay_params.update({
    'foraging_id': foraging_id,
    "mouseID": mouse.id,
    'previous_output_path': behavior_script_output_path,
    "task": {
        "id": TASK_ID,
        "sub_id": "replay",
        "scripts_hash": commit_hash,
    },
})
experiment.camstim_script_path = local_replay_script.as_posix()
experiment.camstim_params = replay_params
experiment.logging_prefix = "replay"
experiment.run_task()

## Run optotagging script

In [ ]:
experiment.camstim_script_path = local_optotagging_script.as_posix()
opto_params["mouseID"] = mouse.id
experiment.camstim_params = opto_params
experiment.logging_prefix = "optotagging"
experiment.run_task()

## Finalize

In [ ]:
platform_json.workflow_complete_time = npxc.now()

experiment.finalize_services(*experiment.recorders, *experiment.stims)
experiment.validate_services(*experiment.recorders, *experiment.stims)

## Copy data

In [ ]:
experiment.copy_data_files()